<a href="https://colab.research.google.com/github/aliirmak/ATPAD/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import packages
import time
from copy import deepcopy
import json
import pymongo
import os
import datetime
import pandas
import math

In [2]:
#get the username and and password information - You need to run it one time at start
from getpass import getpass

USER = getpass('Enter User Name here: ')
PASS = getpass('Enter Password here: ')
HOST = getpass('Enter HOST here: ')
PORT = getpass('Enter PORT here: ')

Enter User Name here: ··········
Enter Password here: ··········
Enter HOST here: ··········
Enter PORT here: ··········


In [0]:
#set up mongo connection
mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(USER,
                                                         PASS,
                                                         HOST,
                                                         PORT,
                                                         "lahman")

client = pymongo.MongoClient(mongo_url)

In [4]:
# Testing connection!
db = client['lahman']
#now print all collection names
print("collection names: {}".format(db.list_collection_names()))
reading = db['Batting'].find_one()
#print one batting documents
reading["_id"] = str(reading["_id"])
print(reading)

collection names: ['AwardsSharePlayers', 'HallOfFame', 'SeriesPost', 'Batting', 'Master', 'AwardsManagers', 'Managers', 'FieldingOF', 'TeamsHalf', 'TeamsFranchises', 'PitchingPost', 'AwardsPlayers', 'Appearances', 'Pitching', 'AllstarFull', 'Salaries', 'Fielding', 'FieldingPost', 'SchoolsPlayers', 'AwardsShareManagers', 'ManagersHalf', 'BattingPost', 'Teams', 'Schools']
{'_id': '5e4ac8491431cc803cb47824', 'playerID': 'aardsda01', 'yearID': 2004, 'stint': 1, 'teamID': 'SFN', 'lgID': 'NL', 'G': 11, 'G_batting': 11, 'AB': 0, 'R': 0, 'H': 0, '2B': 0, '3B': 0, 'HR': 0, 'RBI': 0, 'SB': 0, 'CS': 0, 'BB': 0, 'SO': 0, 'IBB': 0, 'HBP': 0, 'SH': 0, 'SF': 0, 'GIDP': 0, 'G_old': 11}




1. The number of all stars in allstarfull.
2. The most home runs in a season by a single player (using the batting table).
3. The playerid of the player with the most home runs in a season.
4. The number of leagues in the batting table.
5. Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. Note you will nead to cast hits to get a decimal: cast(h as real)
6. The teamid with the fewest hits in the year 2000 (ie., yearid = '2000'). Return both the teamid, and the number of hits. Note you can use ORDER BY column and LIMIT 1.
7. The teamid in the year 2000 (i.e., yearid = '2000') with the highest average batting average. Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.
8. The number of all stars the giants (teamid = 'SFN') had in 2000.
9. The yearid which the giants had the most all stars.
10. The average salary in year 2000.
11. The number of positions (e.g., catchers, pitchers) that have average salaries greather than 2000000 in yearid 2000. You will need to join fielding with salaries. Also consider using a HAVING clause.
12. The number of errors Barry Bonds had in 2000.
13. The average salary of all stars in 2000.
14. The average salary of non-all stars in 2000.

In [190]:
# The number of errors Barry Bonds had in 2000.
# q12 = 'select sum(e) from fielding where playerid = \'bondsba01\' and yearid=\'2000\''

db = client['lahman']
col = db["Fielding"]

results = col.find({"playerID":{"$eq": 'bondsba01'}, "yearID":{"$eq": 2000} })

error = 0
for x in results:
  if x["E"] != '':  
    error = error + x["E"]

print(error)

6


In [0]:
def q1():
  db = client['lahman']
  col = db["AllstarFull"]
  count_result = col.count_documents({})
  return count_result


def q2():
  db = client['lahman']
  col = db["Batting"]
  results = col.find({})

  hr_list = []
  for x in results:
    if x["HR"] != '':
      hr_list.append(x["HR"])

  max_hr = max(hr_list)

  return max_hr


def q3():
  db = client['lahman']
  col = db["Batting"]
  results = col.find({})
  results2 = deepcopy(results)

  hr_list = []
  hr_ind = 0
  for x in results:
    if x["HR"] != '':
      hr_list.append(x["HR"])
    else:
      hr_list.append(0)

  ind_max_hr = hr_list.index(max(hr_list))
  player_max_hr = results2[ind_max_hr]['playerID']

  return player_max_hr


def q4():
  db = client['lahman']
  col = db["Batting"]
  results = col.find({})

  lgid_list = set()
  for x in results:
    if x["lgID"] != '':
      lgid_list.add(x["lgID"])

  no_lgid_list = len(lgid_list)
  return no_lgid_list


def q5():
  db = client['lahman']
  col = db["Batting"]
  results = col.find( {'playerID': {"$eq": 'bondsba01'}} )

  list_ab = []
  list_h = []
  for x in results:
    if x["AB"] != '' and x["H"] != '':
      list_ab.append(x["AB"])
      list_h.append(x["H"])

  list_hab = [h/list_ab[count_h] for count_h, h in enumerate(list_h) ]
  avg_hab = sum(list_hab)/len(list_hab)

  return avg_hab


def q6():
  db = client['lahman']
  col = db["Batting"]
  results = col.find({'yearID': {"$eq": 2000}})

  dict_team = {}

  for x in results:
    if x["teamID"] != '' and x["H"] != '':
      if x["teamID"] in dict_team:
        hval = dict_team[x["teamID"]] + x["H"]
        dict_team.update( {x["teamID"] : hval} )
      else:
        dict_team.update( {x["teamID"] : x["H"]} )

  min_teamid = min(dict_team.keys(), key=(lambda k: dict_team[k]))

  return min_teamid, dict_team[min_teamid]


def q7():
  db = client['lahman']
  col = db["Batting"]
  results = col.find({ 'yearID':{"$eq": 2000}, 'AB':{"$gt": 0} })

  dict_team = {}

  for x in results:
    if x["teamID"] != '' and x["H"] != '' and x["AB"] != '':
      if x["teamID"] in dict_team:
        val_hab  = x["H"]/x["AB"]
        list_hab = dict_team[x["teamID"] ]
        list_hab.append(val_hab)
        dict_team.update( { x["teamID"] : list_hab } )
      else:
        dict_team.update( { x["teamID"] : [x["H"]/x["AB"]] } )

  # min_teamid = min(dict_team.keys(), key=(lambda k: dict_team[k]))

  for key in dict_team:
    avg_hab = sum(dict_team[key])/len(dict_team[key])
    dict_team.update( { key : avg_hab } )

  max_teamid = max(dict_team.keys(), key=(lambda k: dict_team[k]))

  return max_teamid, dict_team[max_teamid]
   

def q8():
  db = client['lahman']
  col = db["AllstarFull"]
  count_result = col.count_documents({ 'yearID':{"$eq": 2000}, 'teamID':{"$eq": 'SFN'} })

  return count_result


def q9():
  db = client['lahman']
  col = db["AllstarFull"]
  results = col.find({ 'teamID':{"$eq": 'SFN'} })

  dict_team = {}
  for x in results:
    if x["yearID"] != '':
      if x["yearID"] in dict_team:
        list_player = dict_team[ x["yearID"] ]
        list_player.add(x['playerID'])
        dict_team.update( { x["yearID"] : list_player } )
      else:
        list_player = {x['playerID']}
        dict_team.update( { x["yearID"] : list_player } )

  for key in dict_team:
    dict_team.update( { key : len(dict_team[key]) } )

  max_yearid = max(dict_team.keys(), key=(lambda k: dict_team[k]))

  return max_yearid


def q10():
  db = client['lahman']
  col = db["Salaries"]
  results = col.find({ 'yearID': {"$eq": 2000} })

  salary = []
  for x in results:
    if x["salary"] != '':  
      salary.append( x["salary"] )

  avg_salary = sum(salary)/len(salary)

  return int(avg_salary)


def q11():
  return 0


def q12():
  db = client['lahman']
  col = db["Fielding"]

  results = col.find({"playerID":{"$eq": 'bondsba01'}, "yearID":{"$eq": 2000} })

  error = 0
  for x in results:
    if x["E"] != '':  
      error = error + x["E"]

  return error


def q13():
  return 0

def q14():
  return 0

In [0]:
def test_q1():
    result_q1 = q1()
    assert result_q1 == 4834

def test_q2():
    result_q2 = q2()
    assert result_q2 == 73

def test_q3():
    result_q3 = q3()
    assert result_q3 == 'bondsba01'

def test_q4():
    result_q4 = q4()
    assert result_q4 == 7

def test_q5():
    result_q5 = q5()
    assert math.isclose(result_q5, 0.29830576286005)

def test_q6():
    result_q6 = q6()
    assert result_q6[0] == 'MIL'
    assert result_q6[1] == 1366

def test_q7():
    result_q7 = q7()
    assert result_q7[0] == 'SEA'
    assert math.isclose(result_q7[1], 0.280799158489005)

def test_q8():
    result_q8 = q8()
    assert result_q8 == 2

def test_q9():
    result_q9 = q9()
    assert result_q9 == 1966

def test_q10():
    result_q10 = q10()
    assert int(result_q10) == 1992984

def test_q11():
    result_q11 = q11()
    assert result_q11 == 3

def test_q12():
    result_q12 = q12()
    assert result_q12 == 6

def test_q13():
    result_q13 = q13()
    assert int(result_q13) == 5388841

def test_q14():
    result_q14 = q14()
    assert int(result_q14) == 1692309

In [194]:
test_q1()
print('Q1 - succesful')
test_q2()
print('Q2 - succesful')
test_q3()
print('Q3 - succesful')
test_q4()
print('Q4 - succesful')
test_q5()
print('Q5 - succesful')
test_q6()
print('Q6 - succesful')
test_q7()
print('Q7 - succesful')
test_q8()
print('Q8 - succesful')
test_q9()
print('Q9 - succesful')
test_q10()
print('Q10 - succesful')
#test_q11()
#print('Q11 - succesful')
test_q12()
print('Q12 - succesful')
test_q13()
print('Q13 - succesful')
test_q14()
print('Q14 - succesful')

print('All asserts are successful!')

Q1 - succesful
Q2 - succesful
Q3 - succesful
Q4 - succesful
Q5 - succesful
Q6 - succesful
Q7 - succesful
Q8 - succesful
Q9 - succesful
Q10 - succesful
Q12 - succesful


AssertionError: ignored